In [1]:
import pandas as pd
import numpy as np
import os
import re
from itertools import chain

In [2]:
folder_path = r'C:\Users\Yijun.Wan\OneDrive - Wolters Kluwer\desktop\version_extract'
%time sf1 = pd.read_csv(os.path.join(folder_path, 'data\SF_Comments.csv'))
%time sf2 = pd.read_csv(os.path.join(folder_path, 'data\SF_Comments_Apr28Jun05.csv'))
%time sf3 = pd.read_csv(os.path.join(folder_path, 'data\SF_Comments_June28_July31.csv'))
%time sf4 = pd.read_csv(os.path.join(folder_path, 'data\SF_Comments_May30_June30.csv'))

Wall time: 7.74 s
Wall time: 126 ms
Wall time: 98.7 ms
Wall time: 154 ms


In [3]:
df = pd.concat([sf1, sf2, sf3, sf4], axis=0, ignore_index = True)
df2 = df[df.comment.notnull()]
comments = df2.comment
[df.shape, len(comments)]
## 1 billion records
len(comments)/1000000

1.046184

In [4]:
folder_path = r'C:\Users\Yijun.Wan\OneDrive - Wolters Kluwer\desktop\version_extract'
lookup_abc = pd.read_csv(os.path.join(folder_path,'lookup\lookup_0820.csv'))
#lookup2 = pd.read_csv(os.path.join(folder_path,'lookup\lookup_0820.csv'))
lookup_abc.head()

,Unnamed: 0,lower_clean,Alphabets,c_id
0,663,rg,RG,10360
1,819,tng,TNG,5659
2,309,fs,FS,3669
3,208,df,DF,1072
4,228,dpm,DPM,1027


In [5]:
lookup_abc.drop('Unnamed: 0', axis=1, inplace=True)
lookup_abc.head()

,lower_clean,Alphabets,c_id
0,rg,RG,10360
1,tng,TNG,5659
2,fs,FS,3669
3,df,DF,1072
4,dpm,DPM,1027


In [6]:
lookup_path = os.path.join(folder_path,'lookup\SomeVersions.xlsx')
table = pd.read_excel(lookup_path, sheet_name = 'Sheet13')
# run only once
versions = table.drop(index=0)['Versions']

### Main

In [7]:
def extract(series, regx = None, show_examples = False):
    # Input data need to be pd.Series
    
    ## define matching criteria
    if regx is None:
        full_regx = '\s.?[A-Za-z_]{3,}.?[0-9]{1,2}[.][0-9]{1,2}[.0-9]*.?\s'
        other_regx = '\s.?[0-9]{1,2}[.\s]?[0-9]{1,2}[.0-9]*.?\s'
        #regular_expression = '\s.?[A-Z]{1}[A-Za-z_]{1,}.?[0-9]{1,2}[.][0-9]{1,2}[.0-9]*.?\s|\s.?[0-9]{1,2}[.][0-9]{1,2}[.0-9]*.?\s'
        regular_expression = r'\s.?[A-Z]{1}[A-Za-z_]{1,}.?[0-9]{1,2}[.][0-9]{1,2}[.0-9]*.?\s|\s[^a-zA-Z]?[0-9]{1,2}[.][0-9]{1,2}[.0-9]*.?\s'
    else:
        regular_expression = regx
    
    new_series = pd.Series([re.sub('\s[vV]ersion is|\s[vV]ersion|[vV]ersion', '', x) for x in series],index = series.index)
    
        # get a match object
        # matches = [re.search(regular_expression ,str(x)) for x in series]
        
    matches = pd.Series([re.findall(regular_expression ,str(x)) for x in new_series],
                        index = new_series.index)

    ind = [len(x)!=0 for x in matches]
    extracted = matches[ind]
    extracted.name = 'Matched Results'

    extract_final = pd.DataFrame(extracted)
    extract_final['Matched Num'] =  extract_final['Matched Results'].apply(len)
    
    
    if show_examples:
        print('Extracted samples:')
        for i, info in extract_final.head(3).iteritems():
            print('-'*20 + '{}'.format(i) + '-'*20)
            print(info)
    # output series
    
    return extract_final




def extract_split(extract_df):
    ## split the lst in 'Matched Results'
    col_id = extract_df.index.values.repeat(extract_df['Matched Num'])
    col_version = list(chain(*list(extract_df['Matched Results'])))
    extract_break = pd.DataFrame({'c_id':col_id, 'extract_info': col_version})
    
    # ma change the original extract_info
    extract_break['extract_info'] = extract_break['extract_info'].apply(lambda x: re.sub('[^a-zA-Z0-9.]', '', x))
    
    return extract_break



def text_split(string):
    # input is a string 
    
    ## clean the unrelated character which will be different from 
    x = re.sub('[^A-Za-z0-9.]+', "", string)
    ind = x[0] in set(['0','1','2','3','4','5','6','7','8','9'])
    if ind:
        return np.nan, x
    else:
                # slice index
        start = re.search('\d',x).span()[0]
        #p1 = re.sub('[^A-Za-z]+', "", x[:start])
        p1 = x[:start]
        p2 = x[start:]
        return p1, p2

    
    
def words_near(comment_string, extract, string, search_num = 10, lower=True):
    
    comment =  re.sub('\s[vV]ersion is|\s[vV]ersion|[vV]ersion', '', comment_string)
    cmt = [re.sub('[^a-zA-Z0-9.]', '', x) for x in re.split('\s|\n', comment)]
    #cmt = re.split(r'\s|\n', comment_string)
    length = len(cmt)
    
    if (extract in cmt):
        p_id = cmt.index(extract)
        
        start = re.search('\d',extract).span()[0]
        new_extract = extract[:start]
        cmt[p_id] = new_extract
        
    elif (string in cmt):
        p_id = cmt.index(string)
    else:
        return {}
    # not enough words before
    l_end = p_id < search_num
    # not enough words after
    r_end = (length - p_id) < 5
    
    if l_end & r_end:
        cmt_sub = cmt
    elif l_end:
        cmt_sub = cmt[:p_id + 5]
        
    elif r_end:
        cmt_sub = cmt[p_id - search_num:]
    else:
        cmt_sub = cmt[p_id - search_num: p_id + 5]
        
    cmt_sub_lower = [re.sub('[^a-z]+', "", x.lower()) for x in cmt_sub]
    cmt_sub = [re.sub('[^A-Za-z]+', "", x) for x in cmt_sub]
    
    if lower:
        return set(cmt_sub_lower)
    else:
        return set(cmt_sub)
    
    
    
    
def find_alphabets(words_near,potential, lower=False):
    if len(words_near)==0:
        return set({})
    words_near = set(words_near)- set({'Was', 'WAs','was', 'be'})
    #s1 = set(['IE', 'BE', 'WAS']).intersection(set(words_near))
    
    if lower:
        a = set([x.lower() for x in words_near])
    else:
        a = set(words_near)
        
    b = set(potential)
    s2 = a.intersection(b)
    
    return s2


In [11]:
def main(comments, lookup):
    regx = None
    show_examples = False
    search_num = 10
    
    
    # series comments
    o1 = extract(comments)
    
    # split list
    o1_break = extract_split(o1)
    
    # split info
    txt_split =  o1_break['extract_info'].apply(text_split)
    o1_break['Alphabets'] = list(txt_split.apply(lambda x: x[0]))
    o1_break['Digits'] = list(txt_split.apply(lambda x: x[1]))
    
    
    ## 'lower': lower alphabets after cleaning
    o1_break['lower'] = o1_break['Alphabets'].dropna().apply(lambda x: re.sub('[^a-z]+', "", x.lower()))
    
    ## no version digits start with '0'
    filt1 = o1_break['Digits'].apply(lambda x: x[0]=='0')  
    mm = o1_break[~filt1]
    
    
    ## 
    m1 = mm[mm['lower'].isin(set(lookup['lower_clean']))]
    m2 = mm[~mm['lower'].isin(set(lookup['lower_clean']))]
    
    ## map back
    m1['versions_1'] = m1['Alphabets']+" "+ m1['Digits']
    mm1 = m1[~m1.duplicated(['c_id', 'Digits', 'lower'], keep='first')]
    mm1_g = mm1.groupby('c_id')['versions_1'].apply(lambda x: ','.join(x)).reset_index()
    
    Comments = pd.DataFrame({'Comment':comments})
    Comments = Comments.join(mm1_g.set_index('c_id'))
    
    
    ## search words
    m2f = m2.copy()
    m2f = m2f[~m2f.duplicated(['c_id', 'Digits'], keep='first')]
    m2f['comment'] = m2['c_id'].map(comments)
    
    m2f_words = m2f[['comment','extract_info', 'Digits']].apply(lambda x: words_near(*x, lower= False), axis=1)
    m2f_words.name = 'words near'
    m2f_new = m2f.join(m2f_words)
    
    mt1 = m2f_new['words near'].apply(lambda x: find_alphabets(x, lookup['lower_clean'], lower=True))
    mt2 = [set(lookup[lookup['lower_clean'].isin(x)]['Alphabets']) for x in mt1]
    m2f_new['Alphabets2'] = mt2
    m2f_new['versions_2'] = m2f_new[['Alphabets2', 'Digits']].apply(lambda x: list(set(map(lambda a:a[0] + ' ' + a[1] , zip(list(x['Alphabets2']),list(np.repeat(x['Digits'],len(x['Alphabets2'])))) ))), axis=1)
    
    ind_fill_digits = m2f_new['versions_2'].apply(lambda x:len(x))==0
    m2f_new['versions_2'][ind_fill_digits] = m2f_new['Digits'][ind_fill_digits].apply(lambda x:[x])
    
    h_lower = m2f_new.groupby('c_id')['versions_2'].apply(list).reset_index()
    h_lower['versions_2'] =  h_lower['versions_2'].apply(lambda x:list(chain(*x)))
    Comments2 = Comments.join(h_lower.set_index('c_id'))
    
    return Comments2

In [12]:
%time oo2 = main(comments, lookup_abc)

C:\Users\Yijun.Wan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Yijun.Wan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Wall time: 4min 40s


In [13]:
oo2.head()

,Comment,versions_1,versions_2
0,STATUS: OPEN => User Response Requested,NaN,NaN
1,8/2/05: m.bolding\nCan this issue be closed or...,NaN,NaN
2,Remain open banks are still requesting some ty...,NaN,NaN
3,USER ESCALATION >> HIGH,NaN,NaN
4,USER ESCALATION >> Emergency/Production Impact,NaN,NaN


In [14]:
ind_2 = oo2['versions_2'].isnull()
ind_1 = oo2['versions_1'].isnull()

df3 = oo2[~(ind_1 & ind_2)]
df3.head()

,Comment,versions_1,versions_2
47,"<FONT COLOR=""""000000""""><B>From: Wood, Paul [ma...",NaN,"[FIRE 2.20, FIRE 7.30.1]"
64,=> http://fsahandbook.info/FSA/html/handbook/B...,"CA 2.2,BIPRU 14.2.1,BIPRU 14.2.13,BIPRU 14.3.,...",NaN
81,FYI: we still go for solution 2. Solution 1 w...,"BIPRU 14.2.2,BIPRU 14.2.13,BIPRU 14.4,BIPRU 14.3",NaN
87,"Hello Sir,\n Request to give us the e...","FRS 1.3.8,FRR 2.6.8",NaN
94,"Philip,\nis this on a 3.0 database? This check...",NaN,"[3.0, 3.0s]"


In [ ]:
df3.to_csv(r'C:\Users\Yijun.Wan\OneDrive - Wolters Kluwer\desktop\version_extract\output\0820_versions_check.csv')

In [ ]:
77042/df3.shape[0]